In [ ]:
from pathlib import Path
import pprint

# run pip install -e . in the root directory to install this package
from stacbuilder import (
    build_collection,
    build_grouped_collections,
    list_asset_metadata,
    list_input_files,
    list_stac_items,
    load_collection,
    validate_collection,
)

In [ ]:
# Collection configuration
catalog_version = "v0.2"
collection_config_path = Path("config-collection.json")

# Input Paths
tiff_input_path = Path("/data/users/Public/emile.sonneveld/ERA5-Land-monthly-averaged-data-ANIN/tiff_collection/")
tiffs_glob = "*/*/*/*.tif"

# Output Paths
output_path = Path("results")
test_output_path = output_path / "test" / catalog_version
publish_output_path = Path("/data/users/Public/victor.verhaert/ANINStac") / catalog_version

In [ ]:
# list input files
input_files = list_input_files(glob=tiffs_glob, input_dir=tiff_input_path, max_files=None)
print(f"Found {len(input_files)} input files. 5 first files:")
for i in input_files[:5]:
    print(i)

In [ ]:
# list meta data
asset_metadata = list_asset_metadata(
    collection_config_path=collection_config_path, glob=tiffs_glob, input_dir=tiff_input_path, max_files=1
)
for k in asset_metadata:
    pprint.pprint(k.to_dict())

In [ ]:
# list items
stac_items, failed_files = list_stac_items(
    collection_config_path=collection_config_path, glob=tiffs_glob, input_dir=tiff_input_path, max_files=10
)
print(f"Found {len(stac_items)} STAC items")
if failed_files:
    print(f"Failed files: {failed_files}")
if len(stac_items) > 0:
    print("First stac item:")
    pprint.pprint(stac_items[0].to_dict())

In [ ]:
stac_items[0]

In [ ]:
# build grouped collection
build_grouped_collections(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=publish_output_path,
)

In [ ]:
# build collection
build_collection(
    collection_config_path=collection_config_path,
    glob=tiffs_glob,
    input_dir=tiff_input_path,
    output_dir=test_output_path,
)

In [ ]:
# show collection
load_collection(collection_file=test_output_path / "collection.json")

In [ ]:
# validate collection
validate_collection(
    collection_file=test_output_path / "collection.json",
)

In [ ]:
import openeo

connection = openeo.connect("https://openeo.vito.be")
connection.authenticate_oidc()

stac_col = connection.load_stac(
    url="/data/users/Public/victor.verhaert/ANINStac/v0.2/collection.json",
    temporal_extent=["2000-01-01", "2000-12-31"],
    spatial_extent={"west": 20, "south": -30.0, "east": 21, "north": -29.0},
)
stac_load_job = stac_col.create_job(
    "GTiff",
    title="load stac",
)
stac_load_job.start()